In [ ]:
############################################## Setup and Data Preparation #######################################

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import matplotlib.pyplot as plt

# Load your dataset
rawdata = pd.read_excel("data.xlsx", parse_dates=["t"], index_col="t")

# Generate the variable of GDP growth

# Calculate GDP growth as the percentage change in GDP
rawdata['gdpgr'] = rawdata['realgdp'].pct_change()

# Replace the first missing value (NaN) with 0
rawdata['gdpgr'].fillna(0, inplace=True)

# Ensure variables of interest are correctly defined
variables = ['gdpgr', 'infl', 'unempl']
data = rawdata[variables]

# Add structural break dummies
rawdata['Break_2003'] = (rawdata.index.year >= 2003).astype(int)
rawdata['Break_2008'] = (rawdata.index.year >= 2008).astype(int)
rawdata['Break_2020'] = (rawdata.index.year >= 2020).astype(int)
rawdata['Break_2022'] = (rawdata.index.year >= 2022).astype(int)

data['Break_2003'] = (data.index.year >= 2003).astype(int)
data['Break_2008'] = (data.index.year >= 2008).astype(int)
data['Break_2020'] = (data.index.year >= 2020).astype(int)
data['Break_2022'] = (data.index.year >= 2022).astype(int)






In [ ]:
################################## Stationarity Testing ####################################

# ADF Test Function
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    return {"ADF Statistic": result[0], "p-value": result[1]}

# Test stationarity for all variables
for var in variables:
    result = adf_test(data[var].dropna())
    print(f"{var}: {result}")


    

In [ ]:
################################ If not stationary ##############################

data_diff = data[variables].diff().dropna()

In [ ]:
####################################### Lag selection and VAR model ######################################

from statsmodels.tsa.stattools import select_order

# Select optimal lags
lag_selection = select_order(data_diff, maxlags=10, trend='c')
optimal_lag = lag_selection.aic
print(f"Optimal Lag: {optimal_lag}")

# Fit VAR Model
var_model = VAR(data_diff)
var_results = var_model.fit(optimal_lag)

# Display summary
print(var_results.summary())

In [ ]:
##################################### Impulse response functions and structural break analysis ######################


# Plot Impulse Response Functions (IRFs)
irf = var_results.irf(10)  # 10-period horizon
irf.plot(orth=False)
plt.show()

# Analyze structural breaks by including dummies as exogenous variables
exog_data = data[['Break_2003', 'Break_2008', 'Break_2020', 'Break_2022']]
var_with_exog = var_model.fit(optimal_lag, exog=exog_data.iloc[1:])
print(var_with_exog.summary())

In [ ]:
################################# Incorporating IV ##########################

from statsmodels.sandbox.regression.gmm import IV2SLS

# Assuming 'GDP_similar' is your IV dataset
iv_data = pd.read_csv("iv_data.csv", parse_dates=["Year"], index_col="Year")

# Merge IV with endogenous variables
merged_data = pd.concat([data, iv_data], axis=1).dropna()

# Perform IV regression for GDP growth
iv_model = IV2SLS(
    merged_data['GDP_growth'], 
    merged_data[['Inflation_rate', 'Unemployment']],  # endogenous regressors
    merged_data[['GDP_similar1', 'GDP_similar2']]    # instruments
)
iv_results = iv_model.fit()
print(iv_results.summary)

In [ ]:
################################### Robustness Check ######################################

# Subsample analysis (e.g., pre-2022 and post-2022)
pre_2022 = data[data.index.year < 2022]
post_2022 = data[data.index.year >= 2022]

# Fit separate VAR models for each sample
var_pre = VAR(pre_2022.diff().dropna()).fit(optimal_lag)
var_post = VAR(post_2022.diff().dropna()).fit(optimal_lag)

# Compare impulse responses
irf_pre = var_pre.irf(10)
irf_post = var_post.irf(10)

irf_pre.plot(orth=False, title="Pre-2022 IRF")
irf_post.plot(orth=False, title="Post-2022 IRF")
plt.show()